In [1]:
# Requires pymongo 3.6.0+
from pymongo import MongoClient
from bson.code import Code

client = MongoClient("mongodb://bigdata-mongodb-04.virtual.uniandes.edu.co:8087/", retryWrites=False)
database = client["Grupo03"]
collection = database["COL_dataset"]

In [8]:
map = Code("function () {"
            'var regex = /[!"#$%&' + "'()*+,-./:;<=>?[\]^_`{|}~]/g;"
            "var palabras = this.reply_or_quote.split(' ');"       
            "palabras.forEach(function(z) {"
            "z = z.replace(regex, '');"
            "z = z.replace(/\\n/g,"");"
            "emit(z.toLowerCase(), 1);"
            "});"
            "}")

In [9]:
reduce = Code("function (key, values) {"
               "  var total = 0;"
               "  for (var i = 0; i < values.length; i++) {"
               "    total += values[i];"
               "  }"
               "  return total;"
               "}")

## Contador de palabras / test

In [10]:
result = database.COL_dataset.map_reduce(map, reduce, "emociones")
#for doc in result.find():
#    print(doc)

In [11]:
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('spanish')) 

collection = database["emociones"]
query = {}
query["_id"] = {
    u"$exists": True
}

projection = {}
projection["_id"] = 1.0
stopwords = {'q', 'si', 'usted', 'tan', 'solo', 'ser', 'bien', 'así', 'mas', 'va', 'van', 'señor', 'hace', 'hacer', 
             'siempre', 'gracias', 'favor', 'puede'}
stop_words.update(stopwords)

cursor = collection.find(query, projection = projection)
try:
    for doc in cursor:
        if (doc['_id'] in stop_words or (doc['_id'][0:1] == '@')):
            collection.delete_one({'_id': doc['_id']})
finally:
    client.close()

In [12]:
client.close()

## N-gramas

In [13]:
import pandas as pd
collection = database["COL_dataset"]

# Created with Studio 3T, the IDE for MongoDB - https://studio3t.com/

query = {}
projection = {}
projection["reply_or_quote"] = 1.0

data = []
cursor = collection.find(query, projection = projection)
try:
    for doc in cursor:
        data.append([doc['reply_or_quote']])
finally:
    client.close()


In [14]:
df = pd.DataFrame(data,columns=['text'])
df.head()

,text
0,@ClaudiaLopez @infopresidencia @Bogota Para cu...
1,@ClaudiaLopez @infopresidencia @Bogota La verd...
2,"@ClaudiaLopez @infopresidencia @Bogota Pero, x..."
3,@ClaudiaLopez @infopresidencia @Bogota Ingreso...
4,@ClaudiaLopez @infopresidencia @Bogota Estimad...


In [ ]:
if (df['text'] == '@')):


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
def get_top_n_words(corpus, n=1,k=1):
    vec = CountVectorizer(ngram_range=(k,k),stop_words = stop_words).fit(corpus)
    print(vec)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [16]:
common_words = get_top_n_words(df['text'], 20,3)
for word, freq in common_words:
    print(word, freq)
df = pd.DataFrame(common_words, columns = ['ReviewText' , 'count'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(3, 3), preprocessor=None,
                stop_words={'a', 'al', 'algo', 'algunas', 'algunos', 'ante',
                            'antes', 'así', 'bien', 'como', 'con', 'contra',
                            'cual', 'cuando', 'de', 'del', 'desde', 'donde',
                            'durante', 'e', 'el', 'ella', 'ellas', 'ellos',
                            'en', 'entre', 'era', 'erais', 'eran', 'eras', ...},
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)
claudialopez infopresidencia bogota 56
trabajar trabajar trabajar 26
revistasemana claudialopez revistadinero 24
claudialopez señora alcaldesa 21
bluradioco claudialopez vieja 19
petrogustavo mucha fuerza 16
petrog

In [17]:
good_pos = ['nc00000', 'nc0n000', 'nc0p000', 'nc0s000', 'np00000']

def ExtractInteresting(sentence, good):
    words = nltk.word_tokenize(sentence)
    interesting = [k for k,v in nltk.pos_tag(words) if v in good]
    print(interesting)
    return(interesting)

In [23]:
import nltk
from nltk.tag import StanfordPOSTagger

tagger="/Users/cristianmartinez/Documents/Anaconda/Twitter/stanford-postagger-full-2018-10-16/models/spanish.tagger"
jar="/Users/cristianmartinez/Documents/Anaconda/Twitter/stanford-postagger-full-2018-10-16/stanford-postagger.jar"

spanish_postagger = StanfordPOSTagger(tagger,jar, encoding='utf8')


sentences = ['@ClaudiaLopez @infopresidencia @Bogota La verdad esa ayuda es una  mentira soy madre cabeza de familia no aparezco en el Sisben y aún Haci no he resubido ningún ayuda se supo e que las personas que no aparecen en el Sisben las localizavan de otra forma @ClaudiaLopez me urge una ayuda','@ClaudiaLopez @infopresidencia @Bogota no se señora alcaldesa si podamos contar con si apoyo ya q debido a esta emergencia mundial no he podido trabajar 😔agradeceria nos tenga en cuenta mi puntaje en el sisben es 18.47 y mi nivel es 1 quedo atenta a algún recibido sra claudia bendiciones y q mi Dios la proteja']

for sent in sentences:

    words = sent.split()
    tagged_words = spanish_postagger.tag(words)

    nouns = []
    palabra_anterior = ''
    for (word, tag) in tagged_words:
        if tag in good_pos:
            if word[0:1] != '@':
#                print(word+' '+tag)
                pareja = palabra_anterior + ' ' +word
                palabra_anterior = word
                nouns.append(pareja)
                print(pareja)

 verdad
verdad ayuda
ayuda mentira
mentira madre
madre cabeza
cabeza familia
familia Sisben
Sisben Haci
Haci ayuda
ayuda personas
personas Sisben
Sisben localizavan
localizavan forma
forma urge
urge ayuda
 señora
señora alcaldesa
alcaldesa apoyo
apoyo emergencia
emergencia 😔agradeceria
😔agradeceria cuenta
cuenta puntaje
puntaje sisben
sisben nivel
nivel sra
sra claudia
claudia bendiciones
bendiciones Dios
